In [31]:
import numpy as np

In [ ]:
class LBM_2_Carlemann:
    def __init__(self):
        # Constants for the D1Q3 lattice (1D, 3 velocities)
        self.w = np.array([2/3, 1/6, 1/6])  # Weights for D1Q3 lattice
        self.e = np.array([0, 1, -1])  # Lattice directions: [0, +1, -1]
        self.c_s = 1 / np.sqrt(3)  # Speed of sound for D1Q3 lattice

        # Parameters
        self.tau = 1.0  # Relaxation time
        self.Nx = 100  # Number of grid points

        # Initialize macroscopic variables: density and velocity field
        self.h = np.ones(self.Nx)  # height field
        self.u = np.zeros(self.Nx)  # Velocity field

        # Initialize distribution functions (f_i), f has 3 directions (D1Q3)
        self.f = np.zeros((self.Nx, 3))  # Distribution functions for D1Q3
        self.feq = np.zeros_like(self.f)  # Equilibrium distribution functions

    # Helper function to compute the equilibrium distribution function for D1Q3
    def equilibrium(self):
        """
        Compute the equilibrium distribution function f_i^{(eq)} based on macroscopic variables.
        """
        feq = np.zeros((self.Nx, 3))
        usqr = self.u**2  # squared velocity
        for m in range(3):
            cu = self.e[m] * self.u  # Dot product of e_i and velocity
            self.feq[:, m] = self.w[m] * self.h * (1 + 3 * cu / self.c_s*2 + 9 / 2 * (cu / self.c_s)**2 - 3 / 2 * usqr / self.c_s*2)
    
    # Helper function to compute moments (density and momentum)
    def compute_moments(self):
        """
        Compute the moments of the distribution function.
        Moment 0: Density, Moment 1: Momentum
        """
        moment_0 = np.sum(self.f, axis=-1)  # Moment 0: density
        moment_1 = np.sum(self.f * self.e, axis=-1)  # Moment 1: momentum (density * velocity)
        return moment_0, moment_1
    

    # Function to build the Carleman linearization matrices
    def carleman_linearization_matrices(self, order=2):
        """
        This function computes the Carleman linearization matrices from the Lattice Boltzmann equation.
        It approximates the non-linear system in terms of moments (density, velocity, etc.).
        """
        # First, compute the equilibrium distribution function
        feq = self.equilibrium()

        # Compute moments: density (moment_0), momentum (moment_1)
        moment_0, moment_1 = self.compute_moments()

        # Now, we form the linearized system matrices for the moments
        # The system of equations for the moments is in the form:
        # Moment derivatives = A * Moment + B
        # Where A is a matrix representing the linear system and B is a bias term.

        # Let's form the Carleman linearization matrices for moments up to the 2nd order
        # For simplicity, we keep the linearization up to the 1st moment (density and velocity)
        
        # Linearized system matrix for the first moment (density)
        A_00 = np.sum(self.w)  # Coefficients for the first moment (density)
        A_01 = np.sum(self.w * self.e)  # Coefficients for the second moment (velocity)
        
        # Linearized system matrix for the second moment (velocity)
        A_10 = np.sum(self.w * self.e)  # Coefficients for the second moment (density)
        A_11 = np.sum(self.w * self.e**2)  # Coefficients for the second moment (momentum)

        # Assemble the Carleman linearization matrices A (for moments) and B (bias terms)
        A = np.array([[A_00, A_01], [A_10, A_11]])
        B = np.array([np.sum(self.w * feq[:, 0]), np.sum(self.w * feq[:, 1])])

        return A, B

In [33]:
Matrix_C = LBM_2_Carlemann()

# Main simulation loop to compute the matrices at each time step
for t in range(1000):
    # Compute the equilibrium distribution function

    # Carleman linearization of the LBE to obtain matrices A and B
    A, B = Matrix_C.carleman_linearization_matrices( order=2)

    # Update the distribution function (based on the LBE and relaxation time tau)
    Matrix_C.f += (1 / Matrix_C.tau) * (Matrix_C.feq - Matrix_C.f)
    
    # Compute macroscopic variables (density and velocity) from the distribution functions
    h = np.sum(Matrix_C.f, axis=-1)  # Compute density
    u = (Matrix_C.f[:, 1] - Matrix_C.f[:, 2]) / h  # Compute velocity

# Print the Carleman linearization matrices after the loop
print("Carleman Linearization Matrix A (Moment Dynamics): \n", A)
print("Carleman Linearization Matrix B (Bias Terms): \n", B)

Carleman Linearization Matrix A (Moment Dynamics): 
 [1. 0.]
Carleman Linearization Matrix B (Bias Terms): 
 [0.         0.33333333]
